#Тема 6. Реализация гибридной модели рекомендаций

###1 часть – общий пример (2 балла)

Цель: научиться использовать библиотеку LightFM для построения гибридной рекомендательной системы, а также оценить качество полученной модели на реальных данных.


#####1.	Установить библиотеку LightFM

In [ ]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831162 sha256=aaa8f54cd629b27c8d39b222c57492459dd32dcac536c30e6fd30a05862d8137
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


#####2.	Выполнить импорт необходимых библиотек

In [ ]:
from lightfm import LightFM

from lightfm.datasets import fetch_movielens # Для использования встроенного в LightFM набора данных Movielens

from lightfm.evaluation import precision_at_k, auc_score


#####3. Подготовка данных

Функция fetch_movielens загружает набор данных Movielens и возвращает словарь, содержащий матрицу оценок, пользовательские функции, характеристики элементов и другую информацию. В построении данной рек. системы нас интересует только матрица оценок.


In [ ]:
data = fetch_movielens(min_rating=5.0);data

{'train': <COOrdinate sparse matrix of dtype 'int32'
 	with 19048 stored elements and shape (943, 1682)>,
 'test': <COOrdinate sparse matrix of dtype 'int32'
 	with 2153 stored elements and shape (943, 1682)>,
 'item_features': <Compressed Sparse Row sparse matrix of dtype 'float32'
 	with 1682 stored elements and shape (1682, 1682)>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

#####4. Реализация гибридной рекомендательной системы

Для реализации рекомендательной системы с lightFM предпочтительным алгоритмом является WARP (Weighted Approximate-Rank Pairwise).


In [ ]:
model = LightFM(loss='warp')

model.fit(data['train'], epochs=30, num_threads=2)


#####5. Генерация рекомендаций

Используем созданную модель для генерации рекомендаций для пользователей, вызвав метод recommend. Метод recommend принимает два аргумента: user_ids и item_ids . Мы можем генерировать рекомендации для одного пользователя или нескольких пользователей одновременно. Мы также можем указать количество рекомендаций для генерации. Следующий код генерирует 10 рекомендаций для пользователя 3.


In [ ]:
import numpy as np

user_id = 3

n_items = data['train'].shape[1]

recommendations = model.predict(user_id, np.arange(n_items))

top_items = np.argsort(-recommendations)[:10]
top_items

array([747, 312, 331, 287, 326, 244, 299, 314, 327, 257])

#####6. Оценка рекомендательной системы

Оцените производительность модели, используя метрики precision at k и AUC. Метрика precision at k измеряет процент рекомендаций, которые были релевантны пользователю, из числа k лучших рекомендаций. Метрика AUC измеряет площадь под кривой рабочей характеристики приемника (ROC), которая показывает частоту истинных положительных срабатываний по сравнению с частотой ложных положительных срабатываний.
Мы можем использовать библиотеку LightFM для вычисления этих показателей. Точность из k рекомендаций может быть рассчитана с помощью метода precision_at_k, а AUC может быть рассчитан с помощью метода auc_score.
В этом примере мы обучаем модель на основе матрицы взаимодействий и оцениваем точность по метрикам k и AUC как для обучающего, так и для тестового наборов данных. Мы используем значение 10 для k, что означает, что мы рассматриваем только 10 лучших рекомендаций для каждого пользователя.
Можно также попробовать вариант модели для k = 5.



In [ ]:
train_precision = np.mean(precision_at_k(model, data['train'], k=10, num_threads=2))

train_auc = np.mean(auc_score(model, data['train'], num_threads=2))

# Оцениваем модель на тестовых данных.
test_precision = np.mean(precision_at_k(model, data['test'], k=10, num_threads=2))

test_auc = np.mean(auc_score(model, data['test'], num_threads=2))

print('Train precision: {:.2f}'.format(train_precision))
print('Train AUC: {:.2f}'.format(train_auc))
print('Test precision: {:.2f}'.format(test_precision))
print('Test AUC: {:.2f}'.format(test_auc))


Train precision: 0.35
Train AUC: 0.97
Test precision: 0.04
Test AUC: 0.92


#####7. Анализ результатов
Проанализируйте результат рекомендаций, сделайте выводы.


точность на обучающей выборке (precision@10) составила 0.35, а AUC — 0.97; на тестовой выборке точность оказалась значительно ниже — 0.04 при AUC 0.92.

Высокое значение метрики AUC на обеих выборках свидетельствует о том, что модель хорошо справляется с задачей ранжирования: она способна различать интересные и неинтересные объекты, определяя порядок, в котором элементы будут показаны пользователю. Это важно для систем, где ключевое значение имеет порядок рекомендаций (например, в лентах новостей, товарных подборках или сервисах потокового видео)

Крайне низкое значение точности на тестовой выборке указывает на то, что из десяти рекомендуемых элементов в среднем только один оказывается действительно релевантным для пользователя. Это делает модель малополезной в практических условиях, особенно там, где важно избежать перенасыщения нерелевантными рекомендациями — например, в email рассылках или push уведомлениях

Причинами такого разрыва между метриками AUC и precision могут быть переобучение модели, чрезмерно жёсткий фильтр по рейтингу, разреженность данных или недостаток информации о пользователях и объектах.

Для повышения качества рекомендаций можно использовать дополнительные пользовательские и контентные признаки, а также рассмотреть другие функции потерь, такие как BPR или логистическая, которые могут лучше работать в конкретных условиях

###2 часть – реализация примера с Kaggle (2 балла)

Рассмотрите решение для конкурса рекомендательных систем Career Village
Конкурс проводился на платформе Kaggle https://www.kaggle.com/code/niyamatalmass/lightfm-hybrid-recommendation-system/notebook , также одно из решений рассмотрено в видео https://rutube.ru/video/9228bcac2604cc498989465cbf0edeb5/?r=wd&t=2679 ).
Цель: создать гибридную рекомендательную систему для рекомендации вопросов студентов профессионалам для CareerVillage.org.
Рекомендательная система работает путем подбора вопросов для профессионалов по тегам, которые соответствуют их специальности, тегам вопросов их предыдущих ответов и похожим тегам. Кроме того, система решает некоторые из наиболее часто встречающихся проблем рекомендательных систем, а именно cold-start и другие.
Career Village предоставляет набор данных о профессионалах, вопросах, на которые ответили профессионалы, студентах, студенческих вопросах.
Комментарии к примеру рассмотрены на лекции, см. слайды темы 5.



1. **Загрузка и очистка данных**  
   - Загрузил таблицы: профессионалы, студенты, вопросы, ответы, теги.
   - Удалил NaN, объединил связанные данные (например, теги с вопросами и профилями).

2. **Преобразование признаков**  
   - Преобразовал текстовые ID (UUID) в числовые (требуется для LightFM).
   - Объединил теги профессионала и теги вопросов, на которые он уже отвечал.

3. **Генерация признаков (features)**  
   - Для вопросов: теги.
   - Для профессионалов: интересы + теги вопросов, на которые они отвечали.

4. **Построение взаимодействий**  
   - Каждое взаимодействие = профессионал ответил на вопрос.
   - Присвоены веса (чем меньше ответов на вопрос, тем выше вес).

5. **Обучение модели LightFM**  
   - Использовал модель с параметрами: loss='warp', no_components=150, epochs=5.
   - Учитывал user_features и item_features.

6. **Генерация рекомендаций**  
   - Для каждого профессионала модель предлагает вопросы, которые он ещё не видел, но которые похожи на его интересы и опыт.

7. **Создание классов**  
   - Всё обернуто в классы для переиспользования:
     - подготовка данных,
     - генерация признаков,
     - обучение модели,
     - выдача рекомендаций (в том числе по диапазону дат).


In [14]:
################################################
# Importing necessary library
################################################
import numpy as np
import pandas as pd
import os

# all lightfm imports
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm import cross_validation
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

# imports re for text cleaning
import re
from datetime import datetime, timedelta

# we will ignore pandas warning
import warnings
warnings.filterwarnings('ignore')
import zipfile

In [17]:
zip_path = '/content/drive/MyDrive/реки/lab_5/input.zip'
output_dir = '/content/drive/MyDrive/реки/lab_5'

os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as archive:
    archive.extractall(output_dir)

print(os.listdir(output_dir))

['input.zip', 'input']


In [19]:
nested_zip_dir = '/content/drive/MyDrive/реки/lab_5/input'

for file in os.listdir(nested_zip_dir):
    if file.endswith('.csv.zip'):
        with zipfile.ZipFile(os.path.join(nested_zip_dir, file), 'r') as zip_ref:
            zip_ref.extractall(nested_zip_dir)

print([f for f in os.listdir(nested_zip_dir) if f.endswith('.csv')])

['groups.csv', 'group_memberships.csv', 'question_scores.csv', 'school_memberships.csv', 'tags.csv', 'answers.csv', 'answer_scores.csv', 'comments.csv', 'emails.csv', 'matches.csv', 'professionals.csv', 'questions.csv', 'students.csv', 'tag_questions.csv', 'tag_users.csv']


In [25]:
base_path = '/content/drive/MyDrive/реки/lab_5/input/'

df_professionals = pd.read_csv(base_path + 'professionals.csv', parse_dates=['professionals_date_joined'])
df_students = pd.read_csv(base_path + 'students.csv', parse_dates=['students_date_joined'])
df_questions = pd.read_csv(base_path + 'questions.csv', parse_dates=['questions_date_added'])
df_answers = pd.read_csv(base_path + 'answers.csv', parse_dates=['answers_date_added'])

In [23]:
'''def generate_int_id(dataframe, id_col_name):
    """
    Generate unique integer id for users, questions and answers

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A.
    id_col_name : String
        New integer id's column name.

    Returns
    -------
    Dataframe
        Updated dataframe containing new id column
    """
    new_dataframe=dataframe.assign(
        int_id_col_name=np.arange(len(dataframe))
        ).reset_index(drop=True)
    return new_dataframe.rename(columns={'int_id_col_name': id_col_name})'''

def generate_int_id(dataframe, id_col_name):
    """
    Generate unique integer id for users, questions and answers
    """
    dataframe = dataframe.copy()
    dataframe[id_col_name] = np.arange(len(dataframe))
    return dataframe.reset_index(drop=True)


def create_features(dataframe, features_name, id_col_name):
    """
    Generate features that will be ready for feeding into lightfm

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe which contains features
    features_name : List
        List of feature columns name avaiable in dataframe
    id_col_name: String
        Column name which contains id of the question or
        answer that the features will map to.
        There are two possible values for this variable.
        1. questions_id_num
        2. professionals_id_num

    Returns
    -------
    Pandas Series
        A pandas series containing process features
        that are ready for feed into lightfm.
        The format of each value
        will be (user_id, ['feature_1', 'feature_2', 'feature_3'])
        Ex. -> (1, ['military', 'army', '5'])
    """

    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col_name], features))
    return features



def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A.
    features_name : List
        List of feature columns name avaiable in dataframe.

    Returns
    -------
    List of all features for mapping
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features


def calculate_auc_score(lightfm_model, interactions_matrix,
                        question_features, professional_features):
    """
    Measure the ROC AUC metric for a model.
    A perfect score is 1.0.

    Parameters
    ----------
    lightfm_model: LightFM model
        A fitted lightfm model
    interactions_matrix :
        A lightfm interactions matrix
    question_features, professional_features:
        Lightfm features

    Returns
    -------
    String containing AUC score
    """
    score = auc_score(
        lightfm_model, interactions_matrix,
        item_features=question_features,
        user_features=professional_features,
        num_threads=4).mean()
    return score

In [26]:
df_professionals = generate_int_id(df_professionals, 'professionals_id_num')
df_students = generate_int_id(df_students, 'students_id_num')
df_questions = generate_int_id(df_questions, 'questions_id_num')
df_answers = generate_int_id(df_answers, 'answers_id_num')

In [28]:
df_tags = pd.read_csv(base_path + 'tags.csv')
df_tag_questions = pd.read_csv(base_path + 'tag_questions.csv')
df_tag_users = pd.read_csv(base_path + 'tag_users.csv')
df_question_scores = pd.read_csv(base_path + 'question_scores.csv')
df_students = pd.read_csv(base_path + 'students.csv', parse_dates=['students_date_joined'])


# - пропущенные теги и - #
df_tags = df_tags.dropna()
df_tags['tags_tag_name'] = df_tags['tags_tag_name'].str.replace('#', '', regex=False)

# Связка тегов с вопросами
df_tags_question = df_tag_questions.merge(
    df_tags, how='inner',
    left_on='tag_questions_tag_id', right_on='tags_tag_id')

df_tags_question = df_tags_question.groupby(
    ['tag_questions_question_id'])['tags_tag_name'].apply(','.join).reset_index()

df_tags_question = df_tags_question.rename(columns={'tags_tag_name': 'questions_tag_name'})

# Связка тегов с профессионалами
df_tags_pro = df_tag_users.merge(
    df_tags, how='inner',
    left_on='tag_users_tag_id', right_on='tags_tag_id')

df_tags_pro = df_tags_pro.groupby(
    ['tag_users_user_id'])['tags_tag_name'].apply(','.join).reset_index()

df_tags_pro = df_tags_pro.rename(columns={'tags_tag_name': 'professionals_tag_name'})

# + теги к вопросам
df_questions = df_questions.merge(
    df_tags_question, how='left',
    left_on='questions_id', right_on='tag_questions_question_id')

# + теги к профессионалам
df_professionals = df_professionals.merge(
    df_tags_pro, how='left',
    left_on='professionals_id', right_on='tag_users_user_id')

# + оценки к вопросам
df_questions = df_questions.merge(
    df_question_scores, how='left',
    left_on='questions_id', right_on='id')

# + студентов к вопросам (авторы)
df_questions = df_questions.merge(
    df_students, how='left',
    left_on='questions_author_id', right_on='students_id')

# Финальный мёрдж: ответы + вопросы + профессионалы
df_merge = df_answers.merge(
    df_questions, how='inner',
    left_on='answers_question_id', right_on='questions_id')

df_merge = df_merge.merge(
    df_professionals, how='inner',
    left_on='answers_author_id', right_on='professionals_id')

df_merge = df_merge.merge(
    df_question_scores, how='inner',
    left_on='questions_id', right_on='id')

In [29]:
#######################
# Generate some features for calculates weights
# that will use with interaction matrix
#######################

df_merge['num_of_ans_by_professional'] = df_merge.groupby(['answers_author_id'])['questions_id'].transform('count')
df_merge['num_ans_per_ques'] = df_merge.groupby(['questions_id'])['answers_id'].transform('count')
df_merge['num_tags_professional'] = df_merge['professionals_tag_name'].str.split(",").str.len()
df_merge['num_tags_question'] = df_merge['questions_tag_name'].str.split(",").str.len()

In [30]:
print("Maximum number of answer per question : " + str(df_merge['num_ans_per_ques'].max()))
print("Maximum number of tags per professional : " + str(df_merge['num_tags_professional'].max()))
print("Maximum number of tags per question : " + str(df_merge['num_tags_question'].max()))

Maximum number of answer per question : 58
Maximum number of tags per professional : 82.0
Maximum number of tags per question : 54.0


In [31]:
########################
# Merge professionals previous answered
# questions tags into professionals tags
########################

# select professionals answered questions tags
# and stored as a dataframe
professionals_prev_ans_tags = df_merge[['professionals_id', 'questions_tag_name']]
# drop null values from that
professionals_prev_ans_tags = professionals_prev_ans_tags.dropna()
# because professsionals answers multiple questions,
# we group all of tags of each user into single row
professionals_prev_ans_tags = professionals_prev_ans_tags.groupby(
    ['professionals_id'])['questions_tag_name'].apply(
        ','.join).reset_index()

# drop duplicates tags from each professionals rows
professionals_prev_ans_tags['questions_tag_name'] = (
    professionals_prev_ans_tags['questions_tag_name'].str.split(',').apply(set).str.join(','))

# finally merge the dataframe with professionals dataframe
df_professionals = df_professionals.merge(professionals_prev_ans_tags, how='left', on='professionals_id')

# join professionals tags and their answered tags
# we replace nan values with ""
df_professionals['professional_all_tags'] = (
    df_professionals[['professionals_tag_name', 'questions_tag_name']].apply(
        lambda x: ','.join(x.dropna()),
        axis=1))

In [32]:
# handling null values
df_questions['score'] = df_questions['score'].fillna(0)
df_questions['score'] = df_questions['score'].astype(int)
df_questions['questions_tag_name'] = df_questions['questions_tag_name'].fillna('No Tag')
# remove duplicates tags from each questions
df_questions['questions_tag_name'] = df_questions['questions_tag_name'].str.split(',').apply(set).str.join(',')


# fill nan with 'No Tag' if any
df_professionals['professional_all_tags'] = df_professionals['professional_all_tags'].fillna('No Tag')
# replace "" with "No Tag", because previously we replace nan with ""
df_professionals['professional_all_tags'] = df_professionals['professional_all_tags'].replace('', 'No Tag')
df_professionals['professionals_location'] = df_professionals['professionals_location'].fillna('No Location')
df_professionals['professionals_industry'] = df_professionals['professionals_industry'].fillna('No Industry')

# remove duplicates tags from each professionals
df_professionals['professional_all_tags'] = df_professionals['professional_all_tags'].str.split(',').apply(set).str.join(',')



# remove some null values from df_merge
df_merge['num_ans_per_ques']  = df_merge['num_ans_per_ques'].fillna(0)
df_merge['num_tags_professional'] = df_merge['num_tags_professional'].fillna(0)
df_merge['num_tags_question'] = df_merge['num_tags_question'].fillna(0)

Building model in LightFM

In [33]:
# generating features list for mapping
question_feature_list = generate_feature_list(
    df_questions,
    ['questions_tag_name'])

professional_feature_list = generate_feature_list(
    df_professionals,
    ['professional_all_tags'])

In [34]:
# calculate our weight value
df_merge['total_weights'] = 1 / (
    df_merge['num_ans_per_ques'])


# creating features for feeding into lightfm
df_questions['question_features'] = create_features(
    df_questions, ['questions_tag_name'],
    'questions_id_num')

df_professionals['professional_features'] = create_features(
    df_professionals,
    ['professional_all_tags'],
    'professionals_id_num')

In [35]:
########################
# Dataset building for lightfm
########################

# define our dataset variable
# then we feed unique professionals and questions ids
# and item and professional feature list
# this will create lightfm internel mapping
dataset = Dataset()
dataset.fit(
    set(df_professionals['professionals_id_num']),
    set(df_questions['questions_id_num']),
    item_features=question_feature_list,
    user_features=professional_feature_list)


# now we are building interactions matrix between professionals and quesitons
# we are passing professional and questions id as a tuple
# e.g -> pd.Series((pro_id, question_id), (pro_id, questin_id))
# then we use lightfm build in method for building interactions matrix
df_merge['author_question_id_tuple'] = list(zip(
    df_merge.professionals_id_num, df_merge.questions_id_num, df_merge.total_weights))

interactions, weights = dataset.build_interactions(
    df_merge['author_question_id_tuple'])



# now we are building our questions and professionals features
# in a way that lightfm understand.
# we are using lightfm build in method for building
# questions and professionals features
questions_features = dataset.build_item_features(
    df_questions['question_features'])

professional_features = dataset.build_user_features(
    df_professionals['professional_features'])

In [36]:
################################
# Model building part
################################

# define lightfm model by specifying hyper-parametre
# then fit the model with ineteractions matrix, item and user features
model = LightFM(
    no_components=150,
    learning_rate=0.05,
    loss='warp',
    random_state=2019)

model.fit(
    interactions,
    item_features=questions_features,
    user_features=professional_features, sample_weight=weights,
    epochs=5, num_threads=4, verbose=True)

Epoch: 100%|██████████| 5/5 [01:37<00:00, 19.53s/it]


In [37]:
calculate_auc_score(model, interactions, questions_features, professional_features)

np.float32(0.91398245)

In [38]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

def recommend_questions(professional_ids):

    for professional in professional_ids:
        # print their previous answered question title
        previous_q_id_num = df_merge.loc[df_merge['professionals_id_num'] == professional][:3]['questions_id_num']
        df_previous_questions = df_questions.loc[df_questions['questions_id_num'].isin(previous_q_id_num)]
        print('Professional Id (' + str(professional) + "): Previous Answered Questions")
        display_side_by_side(
            df_previous_questions[['questions_title', 'question_features']],
            df_professionals.loc[df_professionals.professionals_id_num == professional][['professionals_id_num','professionals_tag_name']])

        # predict
        discard_qu_id = df_previous_questions['questions_id_num'].values.tolist()
        df_use_for_prediction = df_questions.loc[~df_questions['questions_id_num'].isin(discard_qu_id)]
        questions_id_for_predict = df_use_for_prediction['questions_id_num'].values.tolist()

        scores = model.predict(
            professional,
            questions_id_for_predict,
            item_features=questions_features,
            user_features=professional_features)

        df_use_for_prediction['scores'] = scores
        df_use_for_prediction = df_use_for_prediction.sort_values(by='scores', ascending=False)[:8]
        print('Professional Id (' + str(professional) + "): Recommended Questions: ")
        display(df_use_for_prediction[['questions_title', 'question_features']])

In [39]:
recommend_questions([1200 ,19897, 3])

Professional Id (1200): Previous Answered Questions


,questions_title,question_features
,professionals_id_num,professionals_tag_name
1200,1200,"dj,entrepreneurship,marketing,advertising,football,analytics,data-analysis,java,python,real-estate,billiards,blackjack,break,display-advertising,hip-hop,online-advertising,strategy,management,team-leadership"


Professional Id (1200): Recommended Questions: 


,questions_title,question_features
19011,How do you get started in starting your own bu...,"(19011, [business, marketing, management])"
20217,What are the most common and uncommon battles ...,"(20217, [business, entrepreneurship])"
7750,Are there any good colleges for learning bus...,"(7750, [business, marketing])"
13698,Is it really hard managing a group of people?,"(13698, [business, entrepreneurship, management])"
9802,Is marketing a good major?,"(9802, [business, marketing])"
8559,Which Business field of study is best suited t...,"(8559, [accounting, business, entrepreneurship..."
18829,What schools are best for entrepreneurship?,"(18829, [business, entrepreneurship])"
10073,How important is a business degree when trying...,"(10073, [business, entrepreneurship])"


Professional Id (19897): Previous Answered Questions


...

,questions_title,question_features
19407,How can you be a successful photographer? What...,"(19407, [photography, art, graphic-design])"
9682,How to get started in animation?,"(9682, [artist, art, animation])"
2310,what is one of best things about being an anim...,"(2310, [artist, art, animation, design])"
19275,how do i get my content (animations) out in th...,"(19275, [art, animation])"
6058,How should you start in the Graphic Design ind...,"(6058, [art, design, graphic-design])"
1203,What is a good app to use for animating?,"(1203, [art, animation])"
1416,Is Competition In The Animation Field Low or H...,"(1416, [art, animation])"
13484,Would a Graphic Design degree be a feesible op...,"(13484, [art, graphic-design])"


Professional Id (3): Previous Answered Questions


,questions_title,question_features
108,what are the top colleges for forensic science?,"(108, [college, forensic, criminal-justice])"
754,Are there any Forensic Sceince programs for High School Juniors/Seniors?,"(754, [forensic, science])"
3016,How can i be a transport company owner?,"(3016, [business, college-majors])"
,professionals_id_num,professionals_tag_name
3,3,NaN


Professional Id (3): Recommended Questions: 


,questions_title,question_features
2423,How long does it take to become a Detective?,"(2423, [lawyer, criminal-justice, police, law-..."
17184,What types of Detectives are there?,"(17184, [lawyer, criminal-justice, police, law..."
9778,I want to be a police officer or a police disp...,"(9778, [police-officer, criminal-justice, poli..."
18872,What majors would fit a law enforcement career?,"(18872, [police, law-enforcement, law])"
16214,"Do you go to college, then B.L.E.T( Basic Law ...","(16214, [police, law-enforcement, law])"
10534,Is there any required college courses to becom...,"(10534, [police, law-enforcement, law])"
1941,What are important characteristics of a lawyer?,"(1941, [law-school, lawyer, law-enforcement, l..."
18947,"Could I go straight into Law Enforcememt, when...","(18947, [police, law-enforcement, law])"


###3 часть – ИНДЗ (4 баллов)

Тема: Реализация гибридной модели рекомендаций на собственных данных

Цель: научиться использовать библиотеку LightFM для построения гибридной рекомендательной системы, которая комбинирует коллаборативную фильтрацию и учёт признаков пользователей и/или товаров, на вашем собственном наборе данных.


In [41]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mariavol","key":"b921c4b7f908d0e79ac3af6065cc89e4"}'}

In [42]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [43]:
!kaggle datasets download -d ruchi798/bookcrossing-dataset -p /content/drive/MyDrive/реки/lab_5

Dataset URL: https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset
License(s): CC0-1.0


In [44]:
zip_path = '/content/drive/MyDrive/реки/lab_5/bookcrossing-dataset.zip'
output_dir = '/content/drive/MyDrive/реки/lab_5'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

In [65]:
base_path = '/content/drive/MyDrive/реки/lab_5/Book reviews/Book reviews/'

books = pd.read_csv(base_path + 'BX_Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')
users = pd.read_csv(base_path + 'BX-Users.csv', sep=';', encoding='latin-1', on_bad_lines='skip')
ratings = pd.read_csv(base_path + 'BX-Book-Ratings.csv', sep=';', encoding='latin-1', on_bad_lines='skip')


In [66]:
print(books.columns)
print(users.columns)
print(ratings.columns)

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Index(['User-ID', 'Location', 'Age'], dtype='object')
Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


#####Ход работы
1.	Подготовка данных:
 - Загрузите ваш набор данных и подготовьте три компонента:
 -- матрицу взаимодействий;
 -- признаки пользователей;
 --	признаки товаров.
 - Если у вас нет явных признаков пользователей или товаров, используйте dummy-признаки (например, идентификаторы).


In [67]:
#Матрица взаимодействий: только положительные оценки
ratings = ratings[ratings['Book-Rating'] > 0]

# - строки без user/book информации
users = users.dropna(subset=['User-ID', 'Location'])
books = books.dropna(subset=['ISBN', 'Book-Author'])

# Огранич выборку (для ускорения обучения)
top_users = ratings['User-ID'].value_counts().head(5000).index
ratings = ratings[ratings['User-ID'].isin(top_users)]

ratings = ratings.merge(users, on='User-ID', how='left')
ratings = ratings.merge(books, on='ISBN', how='left')

In [68]:
dataset = Dataset()

#  список пользователей и товаров + их признаки
user_ids = ratings['User-ID'].astype(str)
item_ids = ratings['ISBN'].astype(str)
user_features_raw = ratings['Location'].fillna('unknown').astype(str)
item_features_raw = ratings['Book-Author'].fillna('unknown').astype(str)

# Fit для dataset - включает dummy id и признаки
dataset.fit(
    users=user_ids,
    items=item_ids,
    user_features=user_features_raw.unique(),
    item_features=item_features_raw.unique()
)

In [58]:
dataframe=ratings[['User-ID', 'ISBN', 'Book-Rating', 'Location', 'Book-Author']]
dataframe.head(5)

,User-ID,ISBN,Book-Rating,Location,Book-Author
0,276822,0060096195,10,"calgary, alberta, canada",Meggin Cabot
1,276822,0141310340,9,"calgary, alberta, canada",Roald Dahl
2,276822,0142302198,10,"calgary, alberta, canada",Laurel Winter
3,276822,0156006065,9,"calgary, alberta, canada",Raymond Smullyan
4,276822,0375821813,9,"calgary, alberta, canada",CARL HIAASEN


#####2.	Обучение модели:
 - 	Используйте алгоритм WARP (Weighted Approximate-Rank Pairwise) для обучения модели.
 - 	Учтите, что LightFM принимает признаки пользователей и товаров в формате разреженных матриц.


In [69]:
ratings['User-ID'] = ratings['User-ID'].astype(str)
ratings['ISBN'] = ratings['ISBN'].astype(str)
ratings['Location'] = ratings['Location'].fillna('unknown').astype(str)
ratings['Book-Author'] = ratings['Book-Author'].fillna('unknown').astype(str)

In [70]:
#Взаимодействия: пара (user, item)
interactions, _ = dataset.build_interactions(
    ((str(row['User-ID']), str(row['ISBN'])) for _, row in ratings.iterrows())
)

#Признаки пользователей
user_features = dataset.build_user_features(
    ((str(row['User-ID']), [str(row['Location'])]) for _, row in ratings.iterrows())
)

#Признаки товаров
item_features = dataset.build_item_features(
    ((str(row['ISBN']), [str(row['Book-Author'])]) for _, row in ratings.iterrows())
)

In [71]:
model = LightFM(loss='warp', random_state=42)
model.fit(
    interactions,
    user_features=user_features,
    item_features=item_features,
    epochs=5,
    num_threads=2
)


In [72]:
auc = auc_score(
    model, interactions,
    user_features=user_features,
    item_features=item_features
).mean()

precision = precision_at_k(
    model, interactions,
    user_features=user_features,
    item_features=item_features,
    k=10
).mean()

In [73]:
auc, precision

(np.float32(0.82465076), np.float32(0.027480002))

#####3.	Генерация рекомендаций:
 - 	Создайте список топ-N рекомендаций для нескольких пользователей.
 - 	Для проверки выберите хотя бы 5 пользователей.


In [74]:
# Словари для отображения ISBN
isbn_to_title = dict(zip(books['ISBN'], books['Book-Title']))
isbn_to_author = dict(zip(books['ISBN'], books['Book-Author']))


def recommend_books(model, user_id, interactions, user_features, item_features, n=5):
    # Все уникальные книги
    all_books = np.array(list(dataset.mapping()[2].keys()))

    # Книги которые пользователь уже оценивал
    user_index = dataset.mapping()[0][user_id]
    known_positives = ratings[ratings['User-ID'] == user_id]['ISBN'].unique()

    #ещё не взаимодействовали
    unknown_books = [isbn for isbn in all_books if isbn not in known_positives]

    scores = model.predict(
        user_ids=user_index,
        item_ids=np.array([dataset.mapping()[2][isbn] for isbn in unknown_books]),
        user_features=user_features,
        item_features=item_features
    )

    top_indices = np.argsort(-scores)[:n]
    top_isbns = [unknown_books[i] for i in top_indices]

    return [(isbn, isbn_to_title.get(isbn, "Unknown"), isbn_to_author.get(isbn, "Unknown")) for isbn in top_isbns]

In [75]:
sample_users = ratings['User-ID'].drop_duplicates().sample(5, random_state=42).tolist()

for uid in sample_users:
    print(f"\n Рекомендации для пользователя: {uid}")

    recommended = recommend_books(model, uid, interactions, user_features, item_features)
    for idx, (isbn, title, author) in enumerate(recommended, 1):
        print(f"{idx}. {title} — {author}")


 Рекомендации для пользователя: 81950
1. Die Weiss Lowin / Contemporary German Lit — Henning Mankell
2. Wilt — Tom Sharpe
3. Die falsche FÃ?Â¤hrte. — Henning Mankell
4. Der Hahn ist tot. Roman. — Ingrid Noll
5. Hunde von Riga. — Henning Mankell

 Рекомендации для пользователя: 141203
1. The New York Trilogy: City of Glass, Ghosts, the Locked Room (Contemporary American Fiction Series) — Paul Auster
2. El Libro de Las Ilusiones — Paul Auster
3. In the Country of Last Things (Contemporary American Fiction) — Paul Auster
4. Timbuktu : A Novel — Paul Auster
5. Wilt — Tom Sharpe

 Рекомендации для пользователя: 144318
1. The Secret Life of Bees — Sue Monk Kidd
2. Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson — MITCH ALBOM
3. The Secret Life of Bees — Sue Monk Kidd
4. House of Sand and Fog — Andre Dubus III
5. The Nanny Diaries: A Novel — Emma McLaughlin

 Рекомендации для пользователя: 57725
1. The Secret Life of Bees — Sue Monk Kidd
2. Tuesdays with Morrie: An 

#####4.	Оценка модели:
 - 	Оцените производительность модели, используя метрики precision@k и AUC.
 - 	Проверьте результаты как на обучающем, так и на тестовом наборах данных.


In [ ]:
auc, precision

(np.float32(0.82465076), np.float32(0.027480002))

In [76]:
from lightfm.cross_validation import random_train_test_split

train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=np.random.RandomState(42))

model.fit(
    train,
    user_features=user_features,
    item_features=item_features,
    epochs=5,
    num_threads=2
)

train_auc = auc_score(model, train, user_features=user_features, item_features=item_features).mean()
train_precision = precision_at_k(model, train, user_features=user_features, item_features=item_features, k=10).mean()

test_auc = auc_score(model, test, user_features=user_features, item_features=item_features).mean()
test_precision = precision_at_k(model, test, user_features=user_features, item_features=item_features, k=10).mean()

In [77]:
train_auc, train_precision

(np.float32(0.8503071), np.float32(0.022880001))

In [78]:
test_auc, test_precision

(np.float32(0.7096001), np.float32(0.0051100785))